In [1]:
from maelzel.core import *

In [2]:
getPreset('?')

QSocketNotifier: Can only be used with threads started with QThread


Preset: .sing  (Simple vowel singing simulation)
    routing=True
  init: gi__formantFreqs__[] fillarray \
            668, 1191, 2428, 3321, 4600, \  ; A
            327, 2157, 2754, 3630, 4600, \  ; E 
            208, 2152, 3128, 3425, 4200, \  ; I
            335, 628, 2689, 3515, 4200, \   ; O
            254, 796, 2515, 3274, 4160      ; U
                       
        gi__formantDbs__[] fillarray   \
            28, 28, 22, 20, 20, \
            15, 25, 24, 20, 23, \
            10, 20, 27, 26, 20, \
            15, 18, 5,  7,  12, \
            12, 10, 6,  5,  12
                                   
        gi__formantBws__[] fillarray   \
            80, 90, 120, 130, 140, \
            60, 100, 120, 150, 200, \
            60, 90, 100, 120, 120, \
            40, 80, 100, 120, 120, \
            50, 60, 170, 180, 200
            
        gi__formantAmps__[] maparray gi__formantDbs__, "ampdb"
        reshapearray gi__formantFreqs__, 5, 5
        reshapearray gi__formantAmps__, 5, 5
        reshapearray gi__formantBws__, 5, 5
  |kx=0.0, ky=0.0, kvibrange=0.25, kvibfreq=4.5, ivibstart=0.5, ipitchlag=0.2|
  ; Simple vowel singing simulation
  ; Args:
  ;   kx: x coordinate, from 0 to 1, A=(0;0), E=(0.5;0.5), I=(1;0), O=(0;1), U=(1;1)
  ;   ky: y coordinate, from 0 to 1, A=(0;0), E=(0.5;0.5), I=(1;0), O=(0;1), U=(1;1)
  ;   kvibrange: vibrato range in semitones
  ;   ivibstart: start time of vibrato
  ;   ivibfreq: vibrato frequency
  ;   ipitchlag: time lag for pitch modifications

  knoVib = lag:k(trighold(changed2(kpitch), ivibstart*0.8), ivibstart*0.2)
  kvibfreq2 = linseg:k(0, ivibstart*0.25, 0, ivibstart*0.75, 1) * randomi:k(0.9, 1.1, 2) * (1 - knoVib) * kvibfreq 
  kvibamount = linseg:k(0, ivibstart*0.2, 0, ivibstart*0.8, 1) * randomi:k(0.9, 1.1, 10)
  kvibsemi = kvibamount * kvibrange
  kvib = oscil:k(kvibsemi/2, kvibfreq2) - kvibsemi/2
  kpitch2 = lag:k(kpitch, ipitchlag) + kvib
  asource = butterlp:a(vco2:a(kamp, mtof(kpitch2)), 5000)
  kcoords[] fillarray 0, 0, 1,       \  ; A
                      0.5, 0.5, 0.3, \  ; E
                      1, 0, 1,       \  ; I
                      0, 1, 1,       \  ; O
                      1, 1, 1           ; U

  kweights[] presetinterp kx, ky, kcoords, 0.2
  kformantFreqs[] weightedsum gi__formantFreqs__, kweights
  kformantBws[]   weightedsum gi__formantBws__, kweights
  kformantAmps[]  weightedsum gi__formantAmps__, kweights
  kformantFreqs poly 5, "lag", kformantFreqs, ipitchlag
  kformantAmps  poly 5, "lag", kformantAmps, ipitchlag
  aformants[] poly 5, "resonx", asource, kformantFreqs, kformantBws, 2, 1
  aformants *= kformantAmps
  aout1 = sumarray(aformants) * 0.1

In [3]:
presetManager.showPresets('*', full=True)


In [2]:
s = playSession(latency=0)




--Csound version 6.19 (double samples) Jan 13 2024
[commit: c53556fbbb69d14554a4a4bd91efa441a1701d70]
libsndfile-1.2.0
sr = 44100.0, kr = 689.062, ksmps = 64
0dBFS level = 1.0, A4 tuning = 442.0
audio buffered in 256 sample-frame blocks
reading 2048-byte blks of shorts from adc:Built-in Audio Pro: (RAW)
writing 512 sample blks of 64-bit floats to dac:Built-in Audio Pro:
SECTION 1:


In [3]:
foo = defPreset('foo', r'''
|ibusL, ibusR|
prints "ibusL: %d, ibusR: %d\n", ibusL, ibusR
aout1 vco2 kamp, kfreq
aout2 vco2 kamp, kfreq+4

busmix ibusL, aout1
busmix ibusR, aout2
''', output=False)

s.defInstr('master', r'''
    |ibusL, ibusR, kgain=1|
    aL busin ibusL
    aR busin ibusR
    outch 1, aL * kgain, 2, aR * kgain
    ''')

Instr(master, ibusL:5=0, ibusR:6=0, kgain:7=1)

In [4]:
busL = s.assignBus()
busR = s.assignBus()


In [5]:
master = s.sched('master', args=dict(ibusL=busL, ibusR=busR), priority=2)

In [6]:
Note(60.5, 30).play(instr='foo', gain=0.01, ibusL=busL, ibusR=busR)

Button(description='Stop', style=ButtonStyle())

ibusL: 0, ibusR: 1


Output()

SynthGroup(n=1)
    Synth(▶ preset:foo=501.0002 start=5.098 dur=30.000 p4=0.5 kpos=0.01 kgain=17 idataidx_=2 inumbps=3 ibplen=1 ichan=0.02 …)

In [9]:
master.set('kgain', 0.5)

In [13]:
master.stop()

In [10]:
foo = s.defInstr('foo', r'''
    |itime, Sfoo|
    ielapsed times
    prints "Sfoo: %s, itime: %f, ielapsed: %f, diff: %f\n", Sfoo, itime, ielapsed, ielapsed - itime 
    turnoff
''', priority=1)

In [15]:
s.sched("foo", 0, 1, args={'Sfoo': 'bar', 'itime': s.engine.realElapsedTime()})

Sfoo: bar, itime: 74.107937, ielapsed: 74.107937, diff: 0.000000


Button(description='Stop', style=ButtonStyle())

Output()

Synth(▶ foo=502.0006 start=74.108 dur=1.000 p4=74.1079 itime=bar)

In [16]:
s.engine._strToIndex

{'cos': 20, 'linear': 21, 'smooth': 22, 'smoother': 23, 'bar': 24}

In [17]:
test = s.defInstr('test', r'''
    |istrid|
    S0 = strget(istrid)
    prints "S0: %s\n", S0
    turnoff
''')

In [18]:
jiji = s.engine.strSet('jiji')

In [19]:
s.sched('test', 0, 1, args={'istrid': jiji})

Button(description='Stop', style=ButtonStyle())

Output()

S0: jiji


Synth(▶ test=503.0002 start=103.771 dur=1.000 p4=25)

In [20]:
test = s.defInstr('test', r'''
    |ichan=0|
    prints "ichan: %d\n", ichan
    Schan = "bar"
    ; Schan strget ichan
    prints "Schan: %s\n", Schan
    a0 oscili 0.1, 1000
    ; chnset a0, Schan
''')


In [21]:
print(s.generateInstrBody(test))

ichan = p5
prints "ichan: %d\n", ichan
Schan = "bar"
; Schan strget ichan
prints "Schan: %s\n", Schan
a0 oscili 0.1, 1000
; chnset a0, Schan
atstop 1, 0.01, 0, p1


In [22]:
ichan = s.engine.strSet('foo')
ichan

26

In [23]:
s.sched('test', args={'ichan': 0})

Button(description='Stop', style=ButtonStyle())

ichan: 0
Schan: bar


Output()

Synth(▶ test=503.0003 start=112.688 dur=-1.000 p4=0)